In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

# from google.colab import drive
# drive.mount('/content/gdrive')

# image_path = "gdrive/My Drive/101_ObjectCategories"
!mkdir -p ../../datasets
!curl http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz --output ../../datasets/caltech101.tar.gz
!tar -xvzf ../../datasets/caltech101.tar.gz --directory ../../datasets
!mv ../../datasets/101_ObjectCategories ../../datasets/caltech101
!rm -rf ../../datasets/caltech101/BACKGROUND_Google

In [0]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import tensorflow as tf
import os
import time
from tensorflow.keras.preprocessing import image
!pip install git+https://github.com/asrdav/classification_models.git


  Cloning https://github.com/asrdav/classification_models.git to /tmp/pip-req-build-7rl54gj_
  Running command git clone -q https://github.com/asrdav/classification_models.git /tmp/pip-req-build-7rl54gj_
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20032 sha256=ececfb3785efe4464274b5814e69b3ae6059c9392c2ddc7dda40dfa46b17c4ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-vd9mva1_/wheels/02/f7/43/0fce138e361fe041ef61ee8883aac00bc7bc105bdf1b7cf6d0
Successfully built image-classifiers


In [0]:
# for keras
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
# model = ResNet50(weights='imagenet', include_top=False,
#                  input_shape=(96, 112, 3))

from classification_models.keras import Classifiers
from keras.layers import Input

ResNet34, preprocess_input = Classifiers.get('resnet34')
model = ResNet34(weights="imagenet", include_top=False, input_tensor=Input(shape=(96, 112, 3)))

In [112]:
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(
        96, 112))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features 

features = extract_features('image_0004.jpg', model)
print(len(features))

6144


In [0]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
def get_file_list(root_dir):
    file_list = []
    counter = 1
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
    return file_list

    

In [0]:
# path to the datasets
root_dir = '../../datasets/caltech101'
filenames = sorted(get_file_list(root_dir))

In [0]:
feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
